# Import ontologies
The ontologies needed in our example are the domain ontologies, the function ontology and the datamodel ontology


In [1]:
from s7.graph import Graph
g = Graph()

# Read the function ontology
g.parse('function.ttl')

# Read the datamodel ontology
g.parse('https://raw.githubusercontent.com/emmo-repo/datamodel-ontology/master/entity.ttl')

# Read the mappings ontology
g.parse('https://raw.githubusercontent.com/emmo-repo/domain-mappings/master/mappings.ttl')


# Define datamodel for reading the datasource


In [2]:
from s7.xlsparser import XLSParser
from s7.models import create_config
from s7.classgen import class_factory

# Import Data Configuration for the specific parser selected
xls_config = create_config('/home/jovyan/work/semantic_interop_story/inhibitor_conf.yml')

# Generate Specific parser from config (combined generic parser with config)
parser = XLSParser(xls_config)g = Graph()

# Represent data importer as an entity
XLSData = class_factory("XLSData", parser.get())

# Show that data can be read from the data source

In [3]:
d = XLSData() # Instance
print (d.sampleType)

['LiCO3', 'LiCO3', 'LiCl', 'LiCl', 'LiCl']


# Map data

In [4]:
from s7.infix import Infix

g.clear()

def maps_to(g, s, o):
    g.append((s,'mapsTo',o))
    g.append((o,'mappedFrom,',s))


mapsTo = Infix(lambda s, o: maps_to(g, s, o))
has    = Infix(lambda s, o: has(g, s, o))

entity_sample_type = (lambda : d.sampleType)
entity_inhibitor_efficiency = (lambda : d.inhibitorEfficiency)
entity_cas_number = (lambda : d.casNumber)

g.append(('myEntity', ':has', entity_sample_type))
g.append((entity_sample_type, ':has', 'myEntity'))
g.append(('myEntity', ':has', entity_inhibitor_efficiency))
g.append((entity_sample_type, ':has', 'myEntity'))
g.append(('myEntity', ':has', entity_cas_number))
g.append((entity_sample_type, ':has', 'myEntity'))

entity_sample_type          |mapsTo| 'x:sampleType'
entity_inhibitor_efficiency |mapsTo| 'x:effiency'
entity_cas_number           |mapsTo| 'x:casNo'

In [5]:
#for s, p, o in g.match(None, 'mapsTo', 'x:sampleType'):
for s, p, o in g.match(None, None, None):
    print (s, p, o)

myEntity :has <function <lambda> at 0x7ffae628d040>
<function <lambda> at 0x7ffae628d040> :has myEntity
myEntity :has <function <lambda> at 0x7ffae628d280>
myEntity :has <function <lambda> at 0x7ffae628d1f0>
<function <lambda> at 0x7ffae628d040> mapsTo x:sampleType
x:sampleType mappedFrom, <function <lambda> at 0x7ffae628d040>
<function <lambda> at 0x7ffae628d280> mapsTo x:effiency
x:effiency mappedFrom, <function <lambda> at 0x7ffae628d280>
<function <lambda> at 0x7ffae628d1f0> mapsTo x:casNo
x:casNo mappedFrom, <function <lambda> at 0x7ffae628d1f0>


# Receiver side

In [6]:
class MyModel:
    samples: str = ""
    efficiency: float = 0

m = MyModel()


m.samples    | mapsTo | 'x.sampleType'
m.efficiency | mapsTo | 'x.efficiency'

# g.path(m.samples, 'myEntity')

In [1]:
from s7.graph import Graph
g = Graph()
g.clear()
g.append(('a', 'to', 'b'))
g.append(('b', 'to', 'c'))
g.append(('d', 'to', 'c'))
g.path('a', 'd')

forward
forward
reverse


['a', 'b', 'c', 'd']

In [8]:
g.path('a', 'd', None, [])